In [1]:
import json

import numpy as np
import argparse
import random
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data
from scipy.io import loadmat

import sys
sys.path.append('/home/yangz2/projects/gmvae_and_gmmvae/pytorch/')
sys.path.append('../../1906-pcgml/pcgml-gmmVae-exp/modules/')
import vglc_with_path_encodings

from model.GMVAE import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/yangz2/projects/modules/')
from log_training_progress import email_training_progress

from tqdm import tqdm_notebook

In [2]:
num_epochs = 10000
num_components = 2

In [3]:
########################################################
## Input Parameters
#########################################################
parser = argparse.ArgumentParser(description='PyTorch Implementation of DGM Clustering')

## Used only in notebooks
parser.add_argument('-f', '--file',
                    help='Path for input file. First line should contain number of lines to search in')

## Dataset
parser.add_argument('--dataset', type=str, choices=['mnist'],
                    default='mnist', help='dataset (default: mnist)')
parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')

## GPU
parser.add_argument('--cuda', type=int, default=0,
                    help='use of cuda (default: 1)')
parser.add_argument('--gpuID', type=int, default=0,
                    help='set gpu id to use (default: 0)')

## Training
parser.add_argument('--epochs', type=int, default=num_epochs,
                    help='number of total epochs to run (default: 200)')
parser.add_argument('--batch_size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--batch_size_val', default=200, type=int,
                    help='mini-batch size of validation (default: 200)')
parser.add_argument('--learning_rate', default=1e-3, type=float,
                    help='learning rate (default: 0.001)')
parser.add_argument('--decay_epoch', default=-1, type=int, 
                    help='Reduces the learning rate every decay_epoch')
parser.add_argument('--lr_decay', default=0.5, type=float,
                    help='Learning rate decay for training (default: 0.5)')

## Architecture
parser.add_argument('--num_classes', type=int, default=num_components,
                    help='number of classes (default: 10)')  # edited for this task
parser.add_argument('--gaussian_size', default=64, type=int,
                    help='gaussian size (default: 64)')
parser.add_argument('--input_size', default=3072, type=int,
                    help='input size (default: 784)')  # edited for this task

## Partition parameters
parser.add_argument('--train_proportion', default=1, type=float,
                    help='proportion of examples to consider for training only (default: 1.0)')

## Gumbel parameters
parser.add_argument('--init_temp', default=1.0, type=float,
                    help='Initial temperature used in gumbel-softmax (recommended 0.5-1.0, default:1.0)')
parser.add_argument('--decay_temp', default=1, type=int, 
                    help='Set 1 to decay gumbel temperature at every epoch (default: 1)')
parser.add_argument('--hard_gumbel', default=0, type=int, 
                    help='Set 1 to use the hard version of gumbel-softmax (default: 1)')
parser.add_argument('--min_temp', default=0.5, type=float, 
                    help='Minimum temperature of gumbel-softmax after annealing (default: 0.5)' )
parser.add_argument('--decay_temp_rate', default=0.013862944, type=float,
                    help='Temperature decay rate at every epoch (default: 0.013862944)')

## Loss function parameters
parser.add_argument('--w_gauss', default=2, type=float,
                    help='weight of gaussian loss (default: 1)')
parser.add_argument('--w_categ', default=1, type=float,
                    help='weight of categorical loss (default: 1)')
parser.add_argument('--w_rec', default=1, type=float,
                    help='weight of reconstruction loss (default: 1)')
parser.add_argument('--rec_type', type=str, choices=['bce', 'mse'],
                    default='bce', help='desired reconstruction loss function (default: bce)')

## Others
parser.add_argument('--verbose', default=0, type=int,
                    help='print extra information at every epoch.(default: 0)')

args = parser.parse_args()

In [4]:
RESULTS_DIR = '/home/yangz2/projects/gmvae_and_gmmvae/results/'
GAME = 'smba'

In [13]:
for i in tqdm_notebook([2, 4, 6, 8, 10, 15, 20, 30, 40, 50]):
    
    args.num_classes = i
    gmvae = GMVAE(args)
    gmvae.network.load_state_dict(torch.load(f'{RESULTS_DIR}/{GAME}_gmvae_trained_pths/{GAME}_gmvae_{i}_10000.pth', map_location=torch.device('cpu')))
    
    elem_per_category = 500
    generated = gmvae.random_generation(elem_per_category)
    
    generated_int = generated.reshape(-1, 16, 16, 12).argmax(axis=-1)
    print(generated_int.shape)
    
    with open(f'{RESULTS_DIR}/{GAME}_gmvae_generations/{GAME}_gmvae_{i}.json', 'w+') as json_f:
        json.dump(generated_int.tolist(), json_f)

(1000, 16, 16)
(2000, 16, 16)
(3000, 16, 16)
(4000, 16, 16)
(5000, 16, 16)


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 92160000 bytes. Error code 12 (Cannot allocate memory)


In [5]:
for i in tqdm_notebook([2, 40]):
    
    args.num_classes = i
    gmvae = GMVAE(args)
    gmvae.network.load_state_dict(torch.load(f'{RESULTS_DIR}/{GAME}_gmvae_trained_pths/{GAME}_gmvae_{i}_10000.pth', map_location=torch.device('cpu')))
    
    elem_per_category = 500
    generated = gmvae.random_generation(elem_per_category)
    
    generated_int = generated.reshape(-1, 16, 16, 12).argmax(axis=-1)
    print(generated_int.shape)
    
    with open(f'{RESULTS_DIR}/{GAME}_gmvae_generations/{GAME}_gmvae_{i}.json', 'w+') as json_f:
        json.dump(generated_int.tolist(), json_f)

(1000, 16, 16)
(20000, 16, 16)

